In [10]:
# imports
import pandas as pd
import numpy as np
import os, time,json
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

In [2]:
# custom functions
def write_json(new_data, filename): 
    "Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/" 
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

def get_movie_with_rating(movie_id):

    #get movie object for the current id
    movie = tmdb.Movies(movie_id)

    #save the .info and .releases dictionaries
    info = movie.info()
    releases = movie.releases()

    #loop through countries in releases
    for country in releases["countries"]:
        #country == US
        if country["iso_3166_1"] == "US":
            #save certification
            info["certification"] = country["certification"]
    return info

In [3]:
# getting login credentials
with open("/Users/chery/.secret/tmdb_api.json", "r") as file:
    login = json.load(file)
## Display the keys of the loaded dict
login.keys()

dict_keys(['api-key'])

In [4]:
# setting API key
tmdb.API_KEY =  login['api-key']

In [5]:
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'title_akas_combined.csv.gz',
 'title_basics_combined.csv.gz',
 'title_ratings_combined.csv.gz']

In [6]:
# Reading in the basics df
dfbasics = pd.read_csv("Data/title_basics_combined.csv.gz")
dfbasics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0068865,movie,Lives of Performers,Lives of Performers,0,2016.0,NaN,90,Drama
3,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
4,tt0082328,movie,Embodiment of Evil,Encarnação do Demônio,0,2008.0,NaN,94,Horror


In [7]:
# creating list of years to get and an errors list
yearsneeded = [2000,2001]
errors = []

In [8]:
# start of outer loop
for YEAR in tqdm_notebook(yearsneeded, desc='YEARS', position=0):
    
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    file_exists = os.path.isfile(JSON_FILE)
    
    # If it does not exist: create it
    if file_exists == False:
        
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
            
    #Saving new year as the current df
    df = dfbasics.loc[dfbasics['startYear']==YEAR].copy()

    # saving movie ids to list
    movie_ids = df['tconst'].copy()
    
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

    #start of inner loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieving the data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Appending the results to existing file using the predefined function in notebook
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])

    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/2761 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/2885 [00:00<?, ?it/s]

In [9]:
print(f"- Total errors encountered: {len(errors)}")

- Total errors encountered: 1060
